In [1]:
import pandas as pd
from tqdm import tqdm
import nltk.data
import re
import spacy
from tqdm import tqdm

nlp = spacy.load('en')
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

In [2]:
# df1 = pd.read_json('Data/New Age/newage_2016_10_01_to_2017_10_01.json')
# df2 = pd.read_json('Data/New Age/newage_2017_10_02_to_2018_04_20.json')

In [3]:
# df = pd.concat([df1, df2], ignore_index=True)

In [4]:
# df.head(1)

In [5]:
# df['news_content'][0]

In [6]:
# doc = nlp(df['news_content'][0])

In [7]:
# location = []
# organization = []
# person = []
# for entity in doc.ents:
#     if (entity.label_ == "LOC" or entity.label_ == "GPE"):
#         location.append(entity.text)
#     elif (entity.label_ == "ORG"):
#         organization.append(entity.text)
#     elif (entity.label_ == "PERSON"):
#         person.append(entity.text)
# print(list(set(location)), list(set(organization)), list(set(person)))

In [8]:
# def find_entities(text):
#     doc = nlp(text)
#     location = []
#     organization = []
#     person = []
#     for entity in doc.ents:
#         if (entity.label_ == "LOC" or entity.label_ == "GPE"):
#             location.append(entity.text)
#         elif (entity.label_ == "ORG"):
#             organization.append(entity.text)
#         elif (entity.label_ == "PERSON"):
#             person.append(entity.text)
#     location = list(set(location))
#     organization = list(set(organization))
#     person = list(set(person))
#     return location, organization, person

In [9]:
# tqdm.pandas()
# df['location_entities'], df['organization_entities'], df['person_entities'] = zip(*df['news_content'].progress_apply(find_entities))

In [10]:
# df.to_pickle('Data/New Age/NewAge_ent.pkl')

In [11]:
df = pd.read_pickle('Data/New Age/NewAge_ent.pkl')

In [12]:
df.head()

,category,date_published,image,news_content,news_id,newspaper,reporter,tags,title,url,location_entities,organization_entities,person_entities
0,Bangladesh,2016-10-01,"{'urls': [], 'captions': []}",Coordinator of 14-party alliance and health mi...,118,New Age,Bangladesh Sangbad Sangstha,"[General elections, Mohammad Nasim, national p...",No national polls before 2019: Nasim,http://www.newagebd.net/article/118/no-nationa...,[Naogaon],"[BSD, BNP, Workers Party, Advocate Enamul Haqu...","[Khairuzzaman Litan, Md Abdul Malek, Samyabadi..."
1,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,"\n\t\t\t\t\t\t\tIn this July 4, 2015 file phot...",83,New Age,Associated Press,"[David Bowie , Rolling Stones]",Concert promoter in $200 million fraud is bip...,http://www.newagebd.net/article/83/concert-pro...,[Indianapolis],"[Worldwide Entertainment Inc., the Rolling Sto...","[Jack Utsick, Charlie Watts, Ronnie Wood, Keit..."
2,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,"\n\t\t\t\t\t\t\tIn this March 7, 2000 photo, R...",86,New Age,Associated Press,"[Oscar Mayer Wiener , Richard Trentlage]",Man who got generations singing Oscar Mayer W...,http://www.newagebd.net/article/86/man-who-got...,"[Fox River Grove, Illinois, Libertyville, Ill....","[Modern Plastic Brooms, Oscar Mayer Wiener, Mc...","[Buckle, Richard Trentlage, Linda Bruun]"
3,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,\n\t\t\t\t\t\t\tA 2015 file photo shows The Gr...,89,New Age,Associated Press,"[Henri Matisse, National Gallery of Art , Pabl...",National Gallery of Art's East Building set t...,http://www.newagebd.net/article/89/national-ga...,"[The Greek Slave, Washington, DC, The East Bui...","[the National Gallery of Art, AP]","[Virginia Dwan, Amedeo Modigliani, Georgia O'K..."
4,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,\n\t\t\t\t\t\t\tThis file photo taken on April...,90,New Age,Agence France-Presse,"[Bob Dylan, European Commission, Michael Jacks...",Sony closes on Michael Jackson's stake in mus...,http://www.newagebd.net/article/90/sony-closes...,"[Jackson, US, Kanye West]","[Music Publishing, The European Commission, So...","[Marvin Gaye, Lady Gaga, Bob Dylan, Michael Ja..."
